In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_210951/1510302545.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Srag

In [14]:
#srag = spark.read.parquet("/dados10t/datalake/standard/vigivac/srag/dbsrag_tb_influd_long_09072022_v8.parquet/")
srag = spark.read.parquet("/dados10t/datalake/raw/vigivac/srag/dbsrag_tb_influd_pandas_idrow_09072022.parquet/")

In [15]:
srag = srag.withColumn('srag_bin', F.lit(1))

# SG

In [16]:
sg = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG/sg_long_09072022_v9.parquet/")

In [17]:
sg = sg.withColumn('sg_bin', F.lit(1))

# Vacinas

In [2]:
vac = spark.read.parquet('/dados10t/datalake/standard/vigivac/vacinacao/vacinacao07072022_pessoasunicas_5doses_v17.parquet/')

In [3]:
vac = vac.selectExpr("*",
    "stack(5,'d1_data_aplicacao',d1_data_aplicacao,'d2_data_aplicacao',d2_data_aplicacao,'d3_data_aplicacao',d3_data_aplicacao,'d4_data_aplicacao',d4_data_aplicacao,'d5_data_aplicacao',d5_data_aplicacao) as (n_vacina,data_aplicacao)")\
.filter(F.col('data_aplicacao').isNotNull())

In [20]:
#vac = spark.read.parquet('/dados10t/datalake/raw/vigivac/vacinacao/vacinacao07072022_v5.parquet/')

In [4]:
vac = vac.withColumn('vac_bin', F.lit(1))

# Merge

In [11]:
srag_sg = srag.unionByName(sg,allowMissingColumns=True )

In [18]:
srag_sg_vac = srag_sg.unionByName(vac, allowMissingColumns=True).checkpoint(eager=True)

23/10/19 13:17:20 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [19]:
srag_sg_vac.printSchema()

root
 |-- id_paciente: string (nullable = true)
 |-- st_notific: string (nullable = true)
 |-- dt_notific: string (nullable = true)
 |-- sem_not: string (nullable = true)
 |-- dt_sin_pri: string (nullable = true)
 |-- sem_pri: string (nullable = true)
 |-- sg_uf_not: string (nullable = true)
 |-- id_regiona: string (nullable = true)
 |-- co_regiona: decimal(10,0) (nullable = true)
 |-- id_municip: string (nullable = true)
 |-- co_mun_not: string (nullable = true)
 |-- id_unidade: string (nullable = true)
 |-- co_uni_not: string (nullable = true)
 |-- cs_sexo: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- nu_idade_n: decimal(10,0) (nullable = true)
 |-- tp_idade: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- cs_gestant: decimal(10,0) (nullable = true)
 |-- cs_raca: string (nullable = true)
 |-- cs_etinia: string (nullable = true)
 |-- cs_escol_n: decimal(10,0) (nullable = true)
 |-- nu_cep: string (nullable = true)
 |-- id_pais: string (nu

In [20]:
srag_sg_vac.count()

531604068

In [22]:
#srag_sg_vac.filter(F.col("id_paciente").isNotNull()).count() #2490196

2490196

In [23]:
srag_sg_vac = srag_sg_vac.withColumn("id_intermediorio_tb_visitas",F.monotonically_increasing_id()) # em um banco com somente esse id vai ser criado o zip with index e ele vai ser para voltar ao banco full

In [24]:
#srag_sg_vac.count() #531604068

531604068

In [25]:
#srag_sg_vac.select("id_intermediorio_tb_visitas").distinct().count() #531604068

531604068

In [26]:
srag_sg_vac.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/bancos_intermediarios/tb_forvisitas_vac_srag_sg_full_id_long.parquet')

In [27]:
srag_sg_vac_ids = srag_sg_vac.select("id_intermediorio_tb_visitas")

In [28]:
import time
start = time.time()

In [29]:
full_id_with_row = srag_sg_vac_ids.rdd.zipWithIndex().toDF().select(F.col("_1.*"),F.col("_2").alias('visit_occurrence_id'))

In [30]:
end = time.time()
print(end - start)

834.0656304359436


In [ ]:
#full_id = srag_sg_vac.withColumn("new_column", F.lit("ABC"))
#w = Window().partitionBy('new_column').orderBy(F.lit('A'))
#full_id_with_row = full_id.withColumn("visit_occurrence_id",F.row_number().over(w)).drop("new_column")

In [31]:
full_id_with_row.columns

['id_intermediorio_tb_visitas', 'visit_occurrence_id']

In [32]:
#full_id_with_row = full_id_with_row.checkpoint(eager = true)

In [33]:
end = time.time()
print(end - start)

834.085613489151


In [34]:
full_id_with_row.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/bancos_intermediarios/tb_forvisitas_id_long_id_int.parquet') #banco com duas variaveis, id long para retornar ao banco full e o novo id das visitas

In [5]:
srag_sg_vac_v2 = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/bancos_intermediarios/tb_forvisitas_vac_srag_sg_full_id_long.parquet')
full_id_with_row_v2 = spark.read.parquet('/dados10t/datalake/standard/omop/data_with_ids/bancos_intermediarios/tb_forvisitas_id_long_id_int.parquet')

In [3]:
srag_sg_vac_v2.count()

531604068

In [4]:
full_id_with_row_v2.count()

531604068

In [5]:
full_id_with_row_v2.select("id_intermediorio_tb_visitas").distinct().count() #531604068

531604068

In [6]:
srag_sg_vac_v2.select("id_intermediorio_tb_visitas").distinct().count() #531604068

531604068

In [6]:
srag_sg_vac_full_variaveis = srag_sg_vac_v2.join(full_id_with_row_v2,['id_intermediorio_tb_visitas'],'inner')

In [8]:
srag_sg_vac_full_variaveis.printSchema()

root
 |-- id_intermediorio_tb_visitas: long (nullable = true)
 |-- id_paciente: string (nullable = true)
 |-- st_notific: string (nullable = true)
 |-- dt_notific: string (nullable = true)
 |-- sem_not: string (nullable = true)
 |-- dt_sin_pri: string (nullable = true)
 |-- sem_pri: string (nullable = true)
 |-- sg_uf_not: string (nullable = true)
 |-- id_regiona: string (nullable = true)
 |-- co_regiona: decimal(10,0) (nullable = true)
 |-- id_municip: string (nullable = true)
 |-- co_mun_not: string (nullable = true)
 |-- id_unidade: string (nullable = true)
 |-- co_uni_not: string (nullable = true)
 |-- cs_sexo: string (nullable = true)
 |-- dt_nasc: string (nullable = true)
 |-- nu_idade_n: decimal(10,0) (nullable = true)
 |-- tp_idade: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- cs_gestant: decimal(10,0) (nullable = true)
 |-- cs_raca: string (nullable = true)
 |-- cs_etinia: string (nullable = true)
 |-- cs_escol_n: decimal(10,0) (nullable = true)
 |-- 

In [9]:
srag_sg_vac_full_variaveis.count()

531604068

# Writting

### srag

In [22]:
srag_id = srag_sg_vac_full_variaveis.filter(F.col('srag_bin') == 1).select(srag.columns + ['visit_occurrence_id']).drop('srag_bin')

In [23]:
srag_id.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/dbsrag_tb_influd_long_09072022_v8-id_omop-v3.parquet')

23/10/19 16:14:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


#### sg

In [24]:
sg_id = srag_sg_vac_full_variaveis.filter(F.col('sg_bin') == 1).select(sg.columns + ['visit_occurrence_id']).drop('sg_bin')

In [25]:
sg_id.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/sg_long_09072022_v9-id_omop-v3.parquet/')

#### vac

In [7]:
vac_id = srag_sg_vac_full_variaveis.filter(F.col('vac_bin') == 1).select(vac.columns + ['visit_occurrence_id']).drop('vac_bin')

In [8]:
vac_id.write.parquet('/dados10t/datalake/standard/omop/data_with_ids/vacinacao07072022_pessoasunicas_5doses_v17-id_omop-v3.parquet/')

23/10/19 16:52:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
